In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from glob import glob
import IPython.display as ipd
import tensorflow as tf
from PIL import Image

2024-03-13 15:00:39.015066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 15:00:39.015148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 15:00:39.181985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/british-bird-song-dataset/BRITISH BIRD SONG METADATA - birdsong_metadata.csv")
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/british-bird-song-dataset/BRITISH BIRD SONG METADATA - birdsong_metadata.csv'

In [ ]:
class_names = df["species"].unique()
print(class_names)

In [ ]:
df["species"].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 25))
ax.barh(df["species"].unique(), df["species"].value_counts())

Getting all the WAV files

In [ ]:
path_to_flac = "/kaggle/input/british-bird-song-dataset/songs/songs"
datafiles = glob(path_to_flac + "*")

Function to generate a spectrogram

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

def generate_spectrogram(file_audio, identifier,):
    audio_data, sample_rate = librosa.load(path_to_flac + "/" + file_audio)
    spec_mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    spec_mel = librosa.power_to_db(spec_mel, ref=np.max)
    figure, axis = plt.subplots(figsize=(15, 5))
    axis.set_title("Mel Spectrogram")
    plt.suptitle(identifier)
    librosa.display.specshow(spec_mel, x_axis='time', y_axis='log', ax=axis)
    return ipd.Audio(path_to_flac + "/" + file_audio, rate=sample_rate)


Generating random spectrogram

In [ ]:
i = np.random.randint(0, df.shape[0])
generate_spectrogram(df.loc[i, "file_id"], df.loc[i, "species"])

In [ ]:
import os
import librosa
import matplotlib.pyplot as plt

# Path to the directory containing FLAC files
directory = '/kaggle/input/british-bird-song-dataset/songs/songs'

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".flac"):
        filepath = os.path.join(directory, filename)
        
        # Load the audio file
        y, sr = librosa.load(filepath)
        
        # Create and plot the spectrogram
        plt.figure(figsize=(10, 4))
        spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        librosa.display.specshow(librosa.power_to_db(spectrogram, ref=np.max), y_axis='mel', fmax=8000, x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram for {}'.format(filename))
        plt.show()



In [ ]:
df.shape

Plotting waveform

In [ ]:
filename = "/kaggle/input/british-bird-song-dataset/songs/songs/xc101936.flac"
audio_data, sample_rate = librosa.load(filename)

plt.plot(audio_data)
plt.title("Waveform")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
filename ="/kaggle/input/british-bird-song-dataset/songs/songs/xc101936.flac"
audio_data, sample_rate = librosa.load(filename)
spectrogram = librosa.stft(audio_data)
spectrogram = np.abs(spectrogram)

plt.imshow(spectrogram, origin='lower', aspect='auto')
#plt.title("Spectrogram")
plt.xlabel("Time (samples)")
plt.ylabel("Frequency")
plt.colorbar()
plt.show()

Preprocessing function for audio

In [ ]:
def process_audio(audio_file):
    audio_data, sample_rate = librosa.load(audio_file, duration=10)
    mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate) 
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec

In [ ]:
filename = '/kaggle/input/british-bird-song-dataset/songs/songs/xc101940.flac'
print(len(process_audio(filename)))
print(len(process_audio(filename)[0]))


Generating a pandas Dataframe to process the data from the CSV and wav files

In [ ]:
df_train = pd.DataFrame({"species": df["species"], "audiopath": path_to_flac +"/"+ df["file_id"]})

# Assuming `process_audio` is a function that generates mel spectrograms
df_train["mel_spec"] = df_train["audiopath"].apply(lambda x: process_audio(x))

# Using factorize to encode class labels
df_train["class"] = df_train["species"].factorize()[0]

Shuffle data

In [ ]:
from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_train.shape

Seperate train and test data

In [ ]:
(train_x, train_y) = df_train["mel_spec"][:100].values, df_train["class"][:100].values
(test_x, test_y) = df_train["mel_spec"][100:].values, df_train["class"][100:].values

In [ ]:
from keras.utils import to_categorical

test_y = to_categorical(test_y, num_classes=len(class_names))
train_y = to_categorical(train_y, num_classes=len(class_names))


In [ ]:
print(class_names)

Normalize data and generate Tensorflow datasets

In [ ]:
train_x = np.stack(train_x[:])
test_x = np.stack(test_x[:])

train_x = tf.keras.utils.normalize(train_x)
test_x = tf.keras.utils.normalize(test_x)

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

In [ ]:
import numpy as np
from skimage.transform import resize

# Assuming all spectrograms should be of size (128, 128)
desired_shape = (128, 128)

# Resize each spectrogram to the desired shape
train_x_resized = [resize(spec, desired_shape) for spec in train_x]
test_x_resized = [resize(spec, desired_shape) for spec in test_x]

# Stack the resized spectrograms
train_x_stacked = np.stack(train_x_resized)
test_x_stacked = np.stack(test_x_resized)

# Now, normalize the data as before
train_x_normalized = tf.keras.utils.normalize(train_x_stacked)
test_x_normalized = tf.keras.utils.normalize(test_x_stacked)

# Then, create the datasets as before
train_dataset = tf.data.Dataset.from_tensor_slices((train_x_normalized, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x_normalized, test_y))


Setting batch sizes

In [ ]:
train_dataset = train_dataset.batch(10)
test_dataset = test_dataset.batch(10)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Reshape, InputLayer, Dropout
from keras.applications import InceptionResNetV2
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras import layers




Setting up model

In [ ]:
model = keras.models.Sequential()
# def stem(inputs):
#model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(128, 130))


model.add(InputLayer(input_shape=(128,431)))
model.add(Reshape((128,431,1)))
model.add(Conv2D(32, (8, 8), input_shape=(128, 431,1), activation='relu'))
model.add(Conv2D(32, (8,8), activation='relu'))
model.add(Conv2D(64, (8,8), activation='relu'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(16, (2,2), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(85, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# model2 = keras.models.Sequential()

# model2.add(InputLayer(input_shape=(128,130)))
# model2.add(Reshape((128,130,1)))
# model2.add(Conv2D(64, (8, 8), input_shape=(128, 130), activation='relu'))
# model2.add(BatchNormalization())
# model2.add(MaxPooling2D(pool_size=(2,2)))
# model2.add(Conv2D(16, (2,2), activation='relu'))
# model2.add(Flatten())
# model2.add(Dropout(0.5))
# model2.add(Dense(128, activation='relu'))
# model2.add(Dense(5, activation='softmax'))

In [ ]:
# model3 = keras.models.Sequential()

# model3.add(InputLayer(input_shape=(128,130)))
# model3.add(Reshape((128,130,1)))
# model3.add(Conv2D(64, (8, 8), input_shape=(128, 130), activation='relu'))
# model3.add(BatchNormalization())
# model3.add(MaxPooling2D(pool_size=(2,2)))
# model3.add(Conv2D(16, (2,2), activation='relu'))
# model3.add(Flatten())
# model3.add(Dropout(0.5))
# model3.add(Dense(128, activation='relu'))
# model3.add(Dense(5, activation='softmax'))

Model compiling

In [ ]:
model.compile('adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), 'accuracy'])

In [ ]:
# model2.compile('adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), 'accuracy'])

In [ ]:
# model3.compile('adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), 'accuracy'])

Model training

In [ ]:
hist = model.fit(train_dataset, epochs=20, validation_data=test_dataset)

In [ ]:
# hist2 = model2.fit(train_dataset, epochs=50, validation_data=test_dataset)

In [ ]:
# hist3 = model3.fit(train_dataset, epochs=20, validation_data=test_dataset)

Model evaluation on accuracy on test dataset

In [ ]:
model.evaluate(test_dataset)

In [ ]:
# model2.evaluate(test_dataset)

In [ ]:
# model3.evaluate(test_dataset)

In [ ]:
y_pred=model.predict(test_x) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_y, axis=1)

In [ ]:
print(y_pred)
print(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
print(cm)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
sns.heatmap(cm, 
            annot=True,
            fmt='g', 
            xticklabels=["Bewick's Wren","Northern Mockingbird","American Robin","Song Sparrow","Northern Cardinal"],
            yticklabels=["Bewick's Wren","Northern Mockingbird","American Robin","Song Sparrow","Northern Cardinal"])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()

In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_accuracy = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

Lots of plots

In [ ]:
plt.title('Loss')
plt.plot(hist.history['loss'], 'r', label='Overall Loss')
plt.plot(hist.history['val_loss'], 'b', label='Loss Value per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# plt.title('Loss')
# plt.plot(hist.history['loss'], 'r', label='Loss for 100 Epoch model')
# plt.plot(hist2.history['loss'], 'b', label='Loss for 50 Epoch model')
# plt.plot(hist3.history['loss'], 'g', label='Loss for 20 Epoch model')
# plt.xlabel('Epoch')
# plt.ylabel('Value')
# plt.legend()
# plt.show()

In [ ]:
plt.title('Precision')
plt.plot(hist.history['precision_13'], 'r', label='Overall Precision')
plt.plot(hist.history['val_precision_13'], 'b', label='Precision Value per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
plt.title('Recall')
plt.plot(hist.history['recall_13'], 'r', label='Overall Recall')
plt.plot(hist.history['val_recall_13'], 'b', label='Recall Value per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
plt.title('Accuracy')
plt.plot(hist.history['accuracy'], 'r', label='Overall Accuracy')
plt.plot(hist.history['val_accuracy'], 'b', label='Accuracy Value per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
audio_file = '/kaggle/input/bird-song-data-set/wavfiles/101308-9.wav'
audio_data, sample_rate = librosa.load(audio_file, duration=3)
mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate) 
mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
mel_spec = tf.expand_dims(mel_spec, axis=0)
bird_list = ["Bewick's Wren", "Northern Mockingbird", "American Robin", "Song Sparrow", "Northern Cardinal"]

test=model.predict(mel_spec)

print(test)

In [ ]:
model.save('my_model.h5')

# Download the model file
import shutil
shutil.move('my_model.h5', '/kaggle/working/my_model.h5')

Generate model file for easy usage later

In [ ]:
# from IPython.display import FileLink

# model.save("saved_model") 
# !zip -r saved_model.zip './saved_model' 
# FileLink(r'./saved_model.zip')